In [1]:
import modelWin_env as modelWin
import discrete_MDP_transition_based_rewards as MDP
import generate_dataset
import numpy as np

In [2]:
def IPS(D):
    horizon = D.shape[1]
    return np.mean(D[:, horizon-1, 5] * np.sum(D[:, :, 2], axis=1))

In [3]:
def stepIPS(D):
    return np.mean(np.sum(D[:, :, 5] * D[:, :, 2], axis=1) )

In [4]:
def WIPS(D):
    horizon = D.shape[1]
    w_H = np.mean(D[:, horizon-1, 5])
    return np.mean(D[:, horizon-1, 5] / w_H * np.sum(D[:, :, 2], axis=1))

In [5]:
def stepWIPS(D):
    n = D.shape[0]
    horizon = D.shape[1]
    w_t = np.mean(D[:, :, 5], axis=0)
    return np.mean(np.sum(D[:, :, 5] / (np.ones((n,1)) * w_t.T) * D[:, :, 2], axis=1))

In [6]:
def DR(D, Q_hat, V_hat):
    #import pdb; pdb.set_trace()
    n = D.shape[0]
    horizon = D.shape[1]
    D_star = np.zeros((n, horizon))

    t = horizon-1
    D_star[:, t] = D[:, t, 5] * (D[:, t, 2] - Q_hat[t, D[:, t, 0].astype(int), D[:, t, 1].astype(int)])
    for t in np.arange(0, horizon-1)[::-1]:
        D_star[:, t] = D[:, t, 5] * (D[:, t, 2] + V_hat[t+1, D[:, t+1, 0].astype(int)]
                                      - Q_hat[t, D[:, t, 0].astype(int), D[:, t, 1].astype(int)])
    return np.mean(V_hat[t, D[:, t, 0].astype(int)] + np.sum(D_star, axis=1))

In [7]:
def WDR(D, Q_hat, V_hat):
    n = D.shape[0]
    horizon = D.shape[1]
    D_star = np.zeros((n, horizon))
    w_t = np.mean(D[:, :, 5], axis=0)
    
    t = horizon-1
    D_star[:, t] = D[:, t, 5] / w_t[t] * (D[:, t, 2] - Q_hat[t, D[:, t, 0].astype(int), D[:, t, 1].astype(int)])
    for t in np.arange(0, horizon-1)[::-1]:
        D_star[:, t] = D[:, t, 5] / w_t[t] * (D[:, t, 2] + V_hat[t+1, D[:, t+1, 0].astype(int)]
                                      - Q_hat[t, D[:, t, 0].astype(int), D[:, t, 1].astype(int)])
    return np.mean(V_hat[t, D[:, t, 0].astype(int)] + np.sum(D_star, axis=1))

In [8]:
def logit(x):
    return np.log(x) - np.log(1 - x)
def expit(x):
    return 1 / (1 + np.exp(-x))

In [9]:
def fit_epsilon(y, Q, weights, epsilon0, max_it):
    """Fit epsilon in the targeting step, using IRLS.
    The argument Q here is the \tilde{Q}(A^(i)_t, S^(i)_t)
    from the write-up"""
    epsilon = epsilon0
    for it in range(max_it):
        ll = -np.sum(weights * (y * np.log(expit(logit(Q) + epsilon))
                                +(1-y) * np.log(1 - expit(logit(Q) + epsilon)) ) )
        grad = -np.sum(weights * (y - expit(logit(Q) + epsilon)) )
        hessian = np.sum( weights * expit(logit(Q) + epsilon)
                         * (1 - expit(logit(Q) + epsilon)) )
        epsilon = epsilon - grad / hessian
        
    return epsilon

In [10]:
def LTMLE(D, Q_hat, V_hat, evaluation_policy_matrix):
    horizon = D.shape[1]; n = D.shape[0]

    V_evaluated = np.zeros(n)

    epsilons = []
    for t in np.arange(0, horizon)[::-1]:
        Delta_t = horizon - t
        R = D[:, t, 2]
        U_tilde = (R + V_evaluated + Delta_t) / (2 * Delta_t)
        Q_t_evaluated = Q_hat[t, D[:, t, 0].astype(int), D[:, t, 1].astype(int)]
        Q_tilde_evaluated = (Q_t_evaluated + Delta_t) / (2 * Delta_t)
        # Targeting step
        epsilons.append(fit_epsilon(y=U_tilde, Q=Q_tilde_evaluated, weights=D[:, t, 5],
                                   epsilon0=0, max_it=5))
        # Evaluate Q_tilde(s_t, a) for a_t = 1, a_t = 2
        Q_tilde_t_star = expit(logit((Q_hat[t, :, :] + Delta_t) / (2 * Delta_t)) + epsilons[-1])
        # Then set V_tilde(s_t) = \sum_{a} Q_tilde(s_t, a) pi_a(a|s_t)
        V_tilde = np.einsum('sa,sa->s', Q_tilde_t_star, evaluation_policy_matrix)
        # Compute V = 2 * Delta_t * (V_tilde - 1)
        V = 2 * Delta_t * (V_tilde - 1/2)
        # Evaluate V
        V_evaluated = V[D[:, t, 0].astype(int)]
    return np.mean(V_evaluated)

In [11]:
# Compare results with the R implementation
n = 100; horizon = 5; n_columns = 8; n_states = 3; n_actions = 2
D = np.genfromtxt('R/data.csv', delimiter=',').reshape((n, horizon, n_columns), order='F')
# Need to subtract 1 from the states and actions indices
D[:, :, 0] -= 1
D[:, :, 1] -= 1
Q_hat = np.genfromtxt('R/Q_hat.csv', delimiter=',').reshape((horizon, n_states, n_actions), order='F')
V_hat = np.genfromtxt('R/V_hat.csv', delimiter=',')
print(IPS(D))
print(WIPS(D))
print(stepIPS(D))
print(stepWIPS(D))
print(DR(D, Q_hat, V_hat))
print(LTMLE(D, Q_hat, V_hat, modelWin.evaluation_policy_matrix))

-0.28125444079999995
-0.4380213758338713
0.02188125109999997
-0.04880679305197754
0.1354181801599999
0.03222255559277132


In [12]:
# Now, try the estimators on an environment
# First, instantiate an environmenent
s0 = 0
env = MDP.discrete_MDP_transition_based_rewards(modelWin.state_transition_matrix,
                                                modelWin.transition_based_rewards)
env.reset(s0)

In [13]:
# Generate some data
n = 10000; horizon = 5
D = generate_dataset.generate_dataset(env, s0, horizon, n,
                                      modelWin.behavior_action_matrix,
                                      modelWin.evaluation_policy_matrix)
D.shape
# columns of D: s, a, r, pi_b, pi_e, rho_t

(10000, 5, 6)

In [14]:
# Get the true value of the policy value
Q0, V0 = env.get_true_Q_and_V_functions(modelWin.evaluation_policy_matrix, 1, horizon)
V0[0,:]

array([0.276, 0.184, 0.184])

In [15]:
print(IPS(D))
print(stepIPS(D))
print(WIPS(D))
print(stepWIPS(D))
print(DR(D, Q0, V0))
print(WDR(D, Q0, V0))

0.288666876295068
0.30934108673038707
0.29705044158721056
0.3161740955037919
0.32034237330683163
0.3211496515689108
